In [4]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [5]:
channels = 1
img_size = 28
latent_dim = 100
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_epochs = 200
sample_interval = 1000
save_dir = "/content/drive/MyDrive/dataset/Result"

In [6]:
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

100%|██████████| 9912422/9912422 [00:00<00:00, 124347872.72it/s]


Extracting ../../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 31052472.14it/s]


Extracting ../../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33789624.51it/s]

Extracting ../../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 14642989.06it/s]


Extracting ../../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw



In [7]:
img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False

In [8]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [15]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [16]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [17]:
discriminator.eval()

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [ ]:
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "/content/drive/MyDrive/dataset/Result/%d.png" % batches_done, nrow=5, normalize=True)

Streaming output truncated to the last 5000 lines.
[Epoch 194/200] [Batch 628/938] [D loss: 0.378394] [G loss: 3.877568]
[Epoch 194/200] [Batch 629/938] [D loss: 0.286255] [G loss: 1.776549]
[Epoch 194/200] [Batch 630/938] [D loss: 0.337266] [G loss: 6.288602]
[Epoch 194/200] [Batch 631/938] [D loss: 0.288224] [G loss: 4.589365]
[Epoch 194/200] [Batch 632/938] [D loss: 0.326098] [G loss: 2.294073]
[Epoch 194/200] [Batch 633/938] [D loss: 0.167458] [G loss: 6.142842]
[Epoch 194/200] [Batch 634/938] [D loss: 0.185667] [G loss: 5.721240]
[Epoch 194/200] [Batch 635/938] [D loss: 0.214289] [G loss: 3.854909]
[Epoch 194/200] [Batch 636/938] [D loss: 0.183494] [G loss: 3.040666]
[Epoch 194/200] [Batch 637/938] [D loss: 0.151574] [G loss: 3.095331]
[Epoch 194/200] [Batch 638/938] [D loss: 0.189496] [G loss: 2.801978]
[Epoch 194/200] [Batch 639/938] [D loss: 0.094611] [G loss: 2.577429]
[Epoch 194/200] [Batch 640/938] [D loss: 0.105033] [G loss: 4.105721]
[Epoch 194/200] [Batch 641/938] [D loss

In [ ]:
torch.save(generator.state_dict(), save_dir + f"/GAN_{n_epochs}.pth")

In [1]:
def PSNR(img1, img2):
    img1 = img1.astype(float)
    img2 = img2.astype(float)
    if img1.shape != img2.shape:
        raise ValueError("Images must have the same dimensions")

    mse = np.mean((img1 - img2) ** 2)

    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr


In [10]:
model = Generator()
model.load_state_dict(torch.load("/content/drive/MyDrive/dataset/Result/GAN_50.pth"))
model.eval()

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=784, bias=True)
    (12): Tanh()
  )
)